In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import csv
import random
import pandas as pd
from tqdm import tqdm
import datetime
import logging
import re
import configparser
import psycopg2
import socket
import sys
from collections import Counter

In [4]:
config = configparser.ConfigParser()
config.read('./config.ini')

# Get the database credentials
db_endpoint = config['database']['host']
db_name = config['database']['database_name']
db_user = config['database']['username']
db_password = config['database']['password']
db_port = int(config['database']['port'])

# Set the timeout for the connection attempt (in seconds)
connection_timeout = 10

try:    
    # Now establish the database connection using psycopg2
    connection = psycopg2.connect(
        host=db_endpoint,
        port = db_port,
        database = db_name,
        user=db_user,
        password=db_password
    )
    print("Connected successfully!")
    

except (socket.timeout, psycopg2.OperationalError) as e:
    if isinstance(e, socket.timeout):
        print("Error: Connection timed out.")
    else:
        print("Error during connection:", e)
    sys.exit(1)  # Terminate the program with a non-zero exit code

Connected successfully!


## Eda para Trading Historic

In [266]:
with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM tbTradingHistoric;")
    columns = [desc[0] for desc in cursor.description]
    data = cursor.fetchall()
    df_trading_historic = pd.DataFrame(data, columns=columns)

In [267]:
df_trading_historic.head()

,Date,Open,High,Low,Close,Adj_Close,Volume,Symbol
0,2023-01-03,151.960007,153.130005,148.470001,150.039993,149.510452,1414300,A
1,2023-01-04,151.649994,153.039993,150.240005,151.669998,151.134705,1247400,A
2,2023-01-05,150.000000,153.070007,148.770004,152.110001,151.573166,1714600,A
3,2023-01-06,154.360001,154.639999,143.009995,147.669998,147.148834,2445000,A
4,2023-01-09,149.690002,151.279999,147.199997,147.470001,146.949524,1269600,A


In [268]:
df_trading_historic.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume',
       'Symbol'],
      dtype='object')

In [269]:
df_trading_historic.shape

(943135, 8)

In [270]:
# Resumen estadístico básico
print(df_trading_historic.describe())

                Open           High            Low          Close  \
count  943135.000000  943135.000000  943135.000000  943135.000000   
mean       36.802673      37.318264      36.309455      36.826225   
std       122.558300     123.983960     121.272416     122.685630   
min         0.000000       0.002800       0.001300       0.001600   
25%         4.380000       4.510000       4.230000       4.370000   
50%        11.350000      11.550000      11.160000      11.350000   
75%        30.990000      31.500000      30.469999      31.000000   
max      6474.529785    6474.529785    6405.000000    6466.000000   

           Adj_Close        Volume  
count  943135.000000  9.431350e+05  
mean       36.760733  1.202514e+06  
std       122.646157  6.073132e+06  
min         0.001600  0.000000e+00  
25%         4.360000  2.720000e+04  
50%        11.300000  1.698000e+05  
75%        30.910000  7.484000e+05  
max      6466.000000  1.894472e+09  


In [271]:
# Información general del DataFrame
print(df_trading_historic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943135 entries, 0 to 943134
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Date       943135 non-null  object 
 1   Open       943135 non-null  float64
 2   High       943135 non-null  float64
 3   Low        943135 non-null  float64
 4   Close      943135 non-null  float64
 5   Adj_Close  943135 non-null  float64
 6   Volume     943135 non-null  int64  
 7   Symbol     943135 non-null  object 
dtypes: float64(5), int64(1), object(2)
memory usage: 57.6+ MB
None


In [272]:
# Contar valores nulos en cada columna
print(df_trading_historic.isnull().sum())

Date         0
Open         0
High         0
Low          0
Close        0
Adj_Close    0
Volume       0
Symbol       0
dtype: int64


In [273]:
#Find the duplicates

duplicate_regist = df_trading_historic.duplicated().sum()
print(f"Cantidad de registros duplicados: {duplicate_regist}")

Cantidad de registros duplicados: 0


In [274]:
#Count the unique symbols in the data
unique_symbols_count = df_trading_historic['Symbol'].nunique()
print(f"Cantidad de símbolos únicos: {unique_symbols_count}")

Cantidad de símbolos únicos: 7080


### Ejemplo de cómo hacer un buen análisis para esta data, en este caso organizamos los tickers dependiendo de su porcentaje de rendimiento en el año

In [276]:
# Step 1: Convert 'Date' column to datetime type
df_trading_historic['Date'] = pd.to_datetime(df_trading_historic['Date'])

# Step 2: Drop rows with missing values in the 'Close' column
df_trading_historic.dropna(subset=['Close'], inplace=True)

# Step 3: Group by symbol and calculate the percentage growth between the first and last closing prices
symbol_growth = df_trading_historic.groupby('Symbol').agg(
    first_close=('Close', 'first'),
    last_close=('Close', 'last'),
    percentage_growth=('Close', lambda x: (x.iloc[-1] - x.iloc[0]) / x.iloc[0] * 100)
).reset_index()

# Step 4: Create a new DataFrame with the calculated percentage growth and closing prices
df_symbol_growth = pd.DataFrame(symbol_growth, columns=['Symbol', 'first_close', 'last_close', 'percentage_growth'])

# Step 5: Sort the new DataFrame in descending order based on the percentage growth
df_symbol_growth = df_symbol_growth.sort_values(by='percentage_growth', ascending=False)

# Print the result
print(df_symbol_growth)

     Symbol  first_close  last_close  percentage_growth
743   BBLGW     0.042000    2.250000        5257.142929
6825   WETG     0.385000    9.110000        2266.233736
5163   PRFX     0.410000    6.840000        1568.292735
1720   CVNA     4.630000   44.020000         850.755926
2158   ENLT     1.960000   18.620001         850.000024
...     ...          ...         ...                ...
4312   MULN     8.000000    0.120000         -98.500000
6170  TBMCR    10.110000    0.143000         -98.585559
4017   MCOM     6.200000    0.080000         -98.709677
5879    SMX    10.190000    0.128000         -98.743866
293    ALLR   362.600006    2.630000         -99.274683

[7080 rows x 4 columns]


## Eda para tickers

In [188]:
with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM tbtickers;")
    columns = [desc[0] for desc in cursor.description]
    data = cursor.fetchall()
    df_tickers = pd.DataFrame(data, columns=columns)

In [189]:
df_tickers.head()

,Symbol,Name,Country,IPO_Year,Volome,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,United States,1999.0,1126393,Industrials,Electrical Products
1,AA,Alcoa Corporation Common Stock,United States,2016.0,1855048,Industrials,Aluminum
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,,2021.0,108237,Industrials,Metal Fabrications
3,AACG,ATA Creativity Global American Depositary Shares,China,2008.0,2719,Consumer Discretionary,Educational Services
4,AACI,Armada Acquisition Corp. I Common Stock,United States,2021.0,14201,Finance,Blank Checks


In [190]:
df_tickers.columns

Index(['Symbol', 'Name', 'Country', 'IPO_Year', 'Volome', 'Sector',
       'Industry'],
      dtype='object')

In [191]:
df_tickers.describe()

,IPO_Year,Volome
count,4364.000000,7.533000e+03
mean,2013.946379,5.541751e+05
std,10.314418,3.353323e+06
min,1925.000000,1.000000e+00
25%,2011.000000,7.051000e+03
50%,2018.000000,4.631400e+04
75%,2021.000000,2.386220e+05
max,2023.000000,1.298889e+08


In [192]:
df_tickers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7533 entries, 0 to 7532
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Symbol    7533 non-null   object 
 1   Name      7533 non-null   object 
 2   Country   7533 non-null   object 
 3   IPO_Year  4364 non-null   float64
 4   Volome    7533 non-null   int64  
 5   Sector    7533 non-null   object 
 6   Industry  7533 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 412.1+ KB


In [193]:
# Contar valores nulos en cada columna
print(df_tickers.isnull().sum())

Symbol         0
Name           0
Country        0
IPO_Year    3169
Volome         0
Sector         0
Industry       0
dtype: int64


Dado lo anterior, se determina que los años null se asignan como 1900

In [196]:
print(df_tickers[df_tickers['Country']==''])

     Symbol                                               Name Country  \
2       AAC  Ares Acquisition Corporation Class A Ordinary ...           
6      AACT  Ares Acquisition Corporation II Class A Ordina...           
55     ACAQ  Athena Consumer Acquisition Corp. Class A Comm...           
95     ACRO  Acropolis Infrastructure Acquisition Corp. Cla...           
111    ADCT                  ADC Therapeutics SA Common Shares           
...     ...                                                ...     ...   
7491   ZETA    Zeta Global Holdings Corp. Class A Common Stock           
7493    ZEV                 Lightning eMotors Inc Common Stock           
7497    ZGN             Ermenegildo Zegna N.V. Ordinary Shares           
7500    ZIM  ZIM Integrated Shipping Services Ltd. Ordinary...           
7512   ZJYL     JIN MEDICAL INTERNATIONAL LTD. Ordinary Shares           

      IPO_Year   Volome                  Sector  \
2       2021.0   108237             Industrials   
6       2

In [197]:
print(df_tickers[df_tickers['Sector']==''])

      Symbol                                               Name  \
6       AACT  Ares Acquisition Corporation II Class A Ordina...   
9     AAIC^B             Arlington Asset Investment Corp 7.00%    
10    AAIC^C  Arlington Asset Investment Corp 8.250% Seies C...   
13     AAM^A  Apollo Asset Management Inc. 6.375% Series A P...   
14     AAM^B  Apollo Asset Management Inc. 6.375% Series B P...   
...      ...                                                ...   
7409  XFLT^A  XAI Octagon Floating Rate & Alternative Income...   
7445    XXII               22nd Century Group Inc. Common Stock   
7450  YCBD^A  cbdMD Inc. 8.0% Series A Cumulative Convertibl...   
7475    YTEN               Yield10 Bioscience Inc. Common Stock   
7479     YVR              Liquid Media Group Ltd. Common Shares   

            Country  IPO_Year   Volome Sector Industry  
6                      2023.0    81968                  
9     United States       NaN       13                  
10    United States     

In [198]:
print(df_tickers[df_tickers['Industry']==''])

      Symbol                                               Name  \
6       AACT  Ares Acquisition Corporation II Class A Ordina...   
9     AAIC^B             Arlington Asset Investment Corp 7.00%    
10    AAIC^C  Arlington Asset Investment Corp 8.250% Seies C...   
13     AAM^A  Apollo Asset Management Inc. 6.375% Series A P...   
14     AAM^B  Apollo Asset Management Inc. 6.375% Series B P...   
...      ...                                                ...   
7409  XFLT^A  XAI Octagon Floating Rate & Alternative Income...   
7445    XXII               22nd Century Group Inc. Common Stock   
7450  YCBD^A  cbdMD Inc. 8.0% Series A Cumulative Convertibl...   
7475    YTEN               Yield10 Bioscience Inc. Common Stock   
7479     YVR              Liquid Media Group Ltd. Common Shares   

            Country  IPO_Year   Volome Sector Industry  
6                      2023.0    81968                  
9     United States       NaN       13                  
10    United States     

Se encontraron valores null para Country, sector e industria, estos valores deberán tener un id 1, y en sus dimensiones contendrán el string 'N/A' para dar una correcta presentación de datos en una interfaz de visualización como BI

In [194]:
#Count the unique symbols in the data
unique_tickers_count = df_tickers['Symbol'].nunique()
print(f"Cantidad de símbolos únicos: {unique_tickers_count}")

Cantidad de símbolos únicos: 7533


In [195]:
#Cantidad de duplicados
symbol_column = df_tickers['Symbol']
symbol_count = Counter(symbol_column)
repeated_symbols = {symbol: count for symbol, count in symbol_count.items() if count > 1}
print(f'Cntidad de Symbol repetidos: {repeated_symbols}')

Cntidad de Symbol repetidos: {}


## EDA Para GIDS Directory

In [277]:
with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM tbgidsdirectory;")
    columns = [desc[0] for desc in cursor.description]
    data = cursor.fetchall()
    df_gids = pd.DataFrame(data, columns=columns)

In [278]:
df_gids.head()

,Symbol,Name,Type
0,COMP,NASDAQ Composite,I
1,INDS,Nasdaq Industrial,I
2,BANK,Nasdaq Bank,I
3,INSR,Nasdaq Insurance,I
4,OFIN,Nasdaq Real Estate and Other Financial Services,I


In [279]:
df_gids.columns

Index(['Symbol', 'Name', 'Type'], dtype='object')

In [280]:
df_gids.describe()

,Symbol,Name,Type
count,9975,9975,9975
unique,9975,9966,3
top,COMP,Nasdaq Nordea SmartBeta Dividend Momentum Beta...,I
freq,1,3,9681


In [281]:
df_gids.shape

(9975, 3)

In [282]:
df_gids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9975 entries, 0 to 9974
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Symbol  9975 non-null   object
 1   Name    9975 non-null   object
 2   Type    9975 non-null   object
dtypes: object(3)
memory usage: 233.9+ KB


In [283]:
# Contar valores nulos en cada columna
print(df_gids.isnull().sum())

Symbol    0
Name      0
Type      0
dtype: int64


In [284]:
#Count the unique symbols in the data
unique_gids_count = df_gids['Symbol'].nunique()
print(f"Cantidad de símbolos únicos: {unique_gids_count}")

Cantidad de símbolos únicos: 9975


In [285]:
from collections import Counter
symbol_column_gids = df_gids['Symbol']
symbol_count_gids = Counter(symbol_column_gids)
repeated_symbols_gids = {symbol: count for symbol, count in symbol_count_gids.items() if count > 1}
print(f'Cantidad de Symbol repetidos: {repeated_symbols_gids}')

Cantidad de Symbol repetidos: {}


In [286]:
from collections import Counter
Name_column_gids = df_gids['Symbol']
Name_count_gids = Counter(Name_column_gids)
repeated_Name_gids = {symbol: count for symbol, count in Name_count_gids.items() if count > 1}
print(f'Cantidad de Names repetidos: {repeated_Name_gids}')

Cantidad de Names repetidos: {}


#### En la data hay un problema, ya que los 'Name' deberían traer valores únicos como 'nasdaq-100' o 'S&P 500', para corregirlo, se hace una lista previa de estos valores para incluirlos en el ETL construyendo las expresiones regulares necesarias

In [167]:
pattern = r'(\bNasdaq-100\b)|(\bNasdaq\b)|(\b[A-Z]+ ETF\b)|(\b[A-Z]+ Muni Bond ETF\b)' +\
        r'|(\bS&P\b)|(\bNASDAQ\b)|(\bThe Capital Strength\b)|(\bSettle\b.*)|(\bFidelity Disruptive\b.*)' + \
        r'|(\bPHLX\b)|(\bThe Capital Strength\b)|(\bOMX\b)|(\bOMRX\b)|(\bFirst North\b)'+\
        r'|(\bDorsey Wright\b)|(\bCompass EMP\b)|(\bGlobal X\b)|(\bOptimal Blue 30Yr\b)'+\
        r'|(\bStrategic Technology & Ecommerce\b)|(\Strategic Hotel & Lodging\b)|(\bStrategic E-Commerce\b)'+\
        r'|(\bStrategic Fintech & Digital Payments\b)|(\bCRSP US\b)'